In [1]:
from dask.distributed import Client, LocalCluster
from dask import delayed, dataframe as dd
import dask

import pandas as pd
import dask.dataframe as dd
import geopandas as gpd
import math
import numpy as np

import momepy
from scripts.tessellation import enclosed_tessellation

import matplotlib.pyplot as plt
import geopandas as gpd

/tmp/ipykernel_5350/1812556411.py:7: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
Daskcluster = LocalCluster(threads_per_worker=2,
                n_workers=8, memory_limit='100GB')

client = Client(Daskcluster)
client

/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44371 instead
  warnings.warn(
2023-04-27 16:39:22,140 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:39:22,142 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:39:22,144 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:39:22,146 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:39:22,148 - distributed.nanny.memory - WARNING - Ignoring provided memory limit 100GB due to system memory limit of 31.31 GiB
2023-04-27 16:39:22,151 - distributed.nanny.memory - WARNING - Ignoring

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44371/status,
Dashboard: http://127.0.0.1:44371/status,Workers: 8
Total threads: 16,Total memory: 250.47 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46513,Workers: 8
Dashboard: http://127.0.0.1:44371/status,Total threads: 16
Started: Just now,Total memory: 250.47 GiB
Comm: tcp://127.0.0.1:41631,Total threads: 2
Dashboard: http://127.0.0.1:33043/status,Memory: 31.31 GiB
Nanny: tcp://127.0.0.1:44839,


In [3]:
streets = gpd.read_parquet("./out/singapore/streets.pq").explode()

buildings = gpd.read_parquet("./out/singapore/buildings.pq")

study_area = gpd.read_parquet("./out/singapore/study_area.pq")

water = gpd.read_parquet("./out/singapore/water.pq")

outlines = gpd.read_parquet("./out/singapore/water_outlines.pq")

/tmp/ipykernel_5350/3071915858.py:1: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  streets = gpd.read_parquet("./out/singapore/streets.pq").explode()


In [4]:
# study_area = gpd.GeoDataFrame(pd.concat( [study_area, streets_buffer], ignore_index=True)).dissolve()

In [5]:
# # Create a figure and axis
# fig, ax = plt.subplots(figsize=(100, 100))

# # Plot study_area in green on the same axis
# study_area.plot(ax=ax, color='green')

# # Plot streets in blue on the same axis
# streets.plot(ax=ax, color='blue')

# # Plot buildings in red on the same axis
# buildings.plot(ax=ax, color='red')

# outlines.plot(ax=ax, color='black')

# # Show the plot
# plt.show()

In [6]:
enclosures = momepy.enclosures(streets.append(outlines), limit= study_area, clip=True)

In [7]:
tessellation = momepy.Tessellation(buildings, unique_id='uID', enclosures=enclosures, use_dask = True).tessellation

# for debugging
tessellation_copy = tessellation

/home/virgilxw/miniconda3/lib/python3.10/site-packages/distributed/client.py:3095: UserWarning: Sending large graph of size 41.94 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
/home/virgilxw/miniconda3/lib/python3.10/site-packages/momepy/coins.py:15: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pyge

In [16]:
# Find the uIDs in tessellation that are not in buildings
tessellation_uids = set(tessellation['uID'])
buildings_uids = set(buildings['uID'])
new_uids = buildings_uids.difference(tessellation_uids)
buildings = buildings[~buildings['uID'].isin(new_uids)]

In [17]:
buildings.to_parquet("./out/singapore/buildings.pq")

In [8]:
tessellation = tessellation_copy

In [97]:
tessellation = tessellation.overlay(water, how="difference")

In [98]:
# Define a delayed function that takes a group as input and returns the filtered and updated tessellation dataframe

@dask.delayed
def process_group(group, building):
        intersection = group.overlay(building, how='intersection')

        # Calculate the area of each polygon
        intersection['area'] = intersection.geometry.area

        # Get the index of the row with the highest area
        largest_overlap = intersection.iloc[intersection['area'].idxmax()].drop(['uID_2', 'area']).rename({'uID_1': 'uID'})
        
        return gpd.GeoDataFrame({'uID': [largest_overlap['uID']], 
                                 'eID' : [largest_overlap['eID']], 
                                 'geometry': [largest_overlap['geometry']]}, 
                                crs=group.crs)


delayed_obj = []

for name, group in tessellation.groupby('uID'):
        if len(group) > 1:
                building = buildings.loc[buildings["uID"] == name]
                delayed_obj.append(process_group(group, building))
                
# Compute the results and concatenate them into a single dataframe
largest_overlaps = dask.compute(delayed_obj)

/home/virgilxw/miniconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:2323: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3414
Right CRS: None

  return geopandas.overlay(
/home/virgilxw/miniconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:2323: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3414
Right CRS: None

  return geopandas.overlay(
/home/virgilxw/miniconda3/lib/python3.10/site-packages/geopandas/geodataframe.py:2323: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:3414
Right CRS: None

  return geopandas.overlay

In [99]:
largest_overlaps = gpd.GeoDataFrame(pd.concat(largest_overlaps[0]), crs=tessellation.crs).reset_index(drop=True)

In [100]:
duplicates = tessellation.duplicated(subset=['uID'], keep=False)

# Drop the duplicated rows
tessellation = tessellation[~duplicates]

tessellation = tessellation.append(largest_overlaps).reset_index(drop=True)

tessellation = tessellation.reset_index(drop=True)

In [101]:
## split streets into 64 smaller dataframe
def split(df, n_splits):
    n_features_per_split = math.ceil(len(df) / n_splits)
    splits = []
    for i, df in enumerate(np.array_split(df, n_splits)):
        splits.append(gpd.GeoDataFrame(df, crs=df.crs, geometry='geometry'))
    return splits

streets_list = split(streets, 64)

In [102]:
@delayed
def apply_buffer(df):
    return df.buffer(3)

delay_objs = [apply_buffer(df) for df in streets_list]
results = dask.compute(*delay_objs)

In [103]:
# unpack results, a tuple of geodataseries, into a geodataframe
roads_buffer = gpd.GeoDataFrame(geometry=pd.concat(results, ignore_index=True)).dissolve()
bridges = roads_buffer.overlay(study_area, how="difference").explode()

/tmp/ipykernel_6095/1859017330.py:3: FutureWarning: Currently, index_parts defaults to True, but in the future, it will default to False to be consistent with Pandas. Use `index_parts=True` to keep the current behavior and True/False to silence the warning.
  bridges = roads_buffer.overlay(study_area, how="difference").explode()


In [104]:
# # Find all rows with NaN or null values in 'uID' column
# mask = tessellation['uID'].isnull()

# # Generate a new sequence of uIDs starting from 2000000
# new_uIDs = np.arange(2000000, 2000000 + mask.sum())

# # Assign new uIDs to the rows with NaN or null values in 'uID' column
# tessellation.loc[mask, 'uID'] = new_uIDs

In [105]:
# tessellation_dropna = tessellation.dropna().reset_index()
# nouID = tessellation[tessellation['uID'].isnull()].dissolve().explode().reset_index()
# tessellation_with_nouID = tessellation_dropna.append(nouID, ignore_index=True)

In [106]:
# # calculate the area of each polygon in the geometry column and add it as a new column
# tessellation['polygon_area'] = tessellation['geometry'].area

# # sort the rows by uID and area
# tessellation = tessellation.sort_values(['polygon_area'], ascending=[True])

# # initialize a counter to keep track of how many times we have seen each uID
# uID_counts = {}

# # iterate over the rows in the GeoDataFrame
# for i, row in tessellation.iterrows():
#     uID = row['uID']
#     if uID in uID_counts:
#         # increment the count for this uID and rename the row's uID accordingly
#         uID_counts[uID] += 1
#         new_uID = uID  + uID_counts[uID]/100
#         tessellation.at[i, 'uID'] = new_uID
        
#         if uID_counts[uID] > 99:
#             raise ValueError("uID: " + str(uID) + " has more than 100 polygons")
#     else:
#         # this is the first time we've seen this uID, so initialize the count to 1
#         uID_counts[uID] = 1

In [107]:
# duplicates = tessellation[tessellation.duplicated(subset='uID', keep=False)]['uID'].unique()

# # Filter the geodataframe to keep only the rows with non-duplicated `uID`s
# tessellation = tessellation[~tessellation['uID'].isin(duplicates)]

# tessellation = tessellation[rows_to_keep]

# tessellation = tessellation.append(largest_overlap).reset_index(drop=True)

# return tessellation

In [108]:
tessellation_bridges = tessellation.copy()

# Iterate through the bridges geodataframe
for index, bridge in bridges.iterrows():
    # Check if the bridge touches any polygon in the tessellation geodataframe
    intersects = tessellation_bridges[tessellation.geometry.intersects(bridge.geometry)]
    if not intersects.empty:
        uid_value = intersects.iloc[0]['uID']
        row = tessellation_bridges.loc[tessellation_bridges['uID'] == uid_value]
        new_geom = row.geometry.union(intersects.iloc[0]["geometry"])
        tessellation_bridges.loc[tessellation_bridges['uID'] == uid_value, 'geometry'] = new_geom

duplicates = tessellation_bridges[tessellation_bridges.duplicated(subset='uID', keep=False)]
if not duplicates.empty:
    print("Duplicate uids found!")
    print(duplicates)
else:
    print("No duplicate uids found.")

No duplicate uids found.


In [109]:
Daskcluster.close()
client.shutdown()

In [ ]:
tessellation.to_parquet("./out/singapore/tessellation.pq")

In [110]:
tessellation_bridges.to_parquet("./out/singapore/tessellation_bridges.pq")

In [111]:
# export geodataframe as shapefile
tessellation.to_parquet("./out/singapore/tessellation.pq")

In [112]:
# buildings.to_parquet("./out/singapore/buildings.pq")

: 